In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Generate a dummy dataset
def generate_dummy_data(n_samples=1000):
    np.random.seed(42)
    data = {
        'age': np.random.randint(18, 90, size=n_samples),
        'cholesterol': np.random.uniform(150, 300, size=n_samples),
        'blood_pressure': np.random.uniform(80, 180, size=n_samples),
        'heart_rate': np.random.uniform(60, 120, size=n_samples),
        'target': np.random.choice([0, 1], size=n_samples)  # Binary classification target
    }
    return pd.DataFrame(data)

In [3]:
# Create dummy dataset
df = generate_dummy_data()

In [4]:
# Inspect the dataset
print("Dataset Overview:")
print(df.head())
print(df.info())
print(df.describe())

Dataset Overview:
   age  cholesterol  blood_pressure  heart_rate  target
0   69   280.049064      154.342379   92.670346       1
1   32   211.197674      141.194051   72.267653       0
2   89   232.758389       90.593071   75.318545       1
3   78   188.083285       95.156937   77.741680       0
4   38   179.416965       89.375692   95.628149       0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             1000 non-null   int64  
 1   cholesterol     1000 non-null   float64
 2   blood_pressure  1000 non-null   float64
 3   heart_rate      1000 non-null   float64
 4   target          1000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 39.2 KB
None
               age  cholesterol  blood_pressure   heart_rate       target
count  1000.000000  1000.000000     1000.000000  1000.000000  1000.000000
mean     52.881000   2

In [5]:
# Convert to PyTorch Dataset class
class DummyDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.features = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        self.targets = torch.tensor(dataframe['target'].values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
# Split dataset into training and validation sets
dataset = DummyDataset(df)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [7]:
# DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [8]:
# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [9]:
# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for features, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}")

Epoch 1/10, Loss: 1.5159
Epoch 2/10, Loss: 0.9432
Epoch 3/10, Loss: 0.7656
Epoch 4/10, Loss: 0.7280
Epoch 5/10, Loss: 0.7236
Epoch 6/10, Loss: 0.7076
Epoch 7/10, Loss: 0.7004
Epoch 8/10, Loss: 0.7049
Epoch 9/10, Loss: 0.6962
Epoch 10/10, Loss: 0.6990


In [11]:
# Validation loop
model.eval()
val_loss = 0
with torch.no_grad():
    for features, targets in val_loader:
        outputs = model(features)
        loss = criterion(outputs, targets)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss/len(val_loader):.4f}")

Validation Loss: 0.6863
